<h1> 바닥 이미지에 도형 합성하기(데이터 만들기) </h1>

In [1]:
import cv2
import numpy as np
import random
import os

# ===============================
# 폴더 경로
# ===============================
IMG_TRAIN = "yolo_dataset/train/images/"
LBL_TRAIN = "yolo_dataset/train/labels/"

IMG_VAL = "yolo_dataset/val/images/"
LBL_VAL = "yolo_dataset/val/labels/"

BG_DIR  = "yolo_dataset/backgrounds/"

# ===============================
# 클래스 정의
# ===============================
# 0 = circle
CLASS_NAMES = ["circle"]

# 클래스별 생성 개수
NUM_PER_CLASS = 200

IMG_SIZE = 640

# ===============================
# 실제 촬영한 배경 불러오기
# ===============================
def load_real_background():
    bg_name = random.choice(os.listdir(BG_DIR))
    bg = cv2.imread(os.path.join(BG_DIR, bg_name))
    bg = cv2.resize(bg, (IMG_SIZE, IMG_SIZE))
    return bg

# ===============================
# 도형 색상 & 채움 여부
# ===============================
colors = [
        (0, 0, 255),      # 빨강 (Red)
        (0, 0, 139),      # 어두운 빨강 (Dark Red)
        (0, 255, 255),    # 노랑 (Yellow)
        (0, 150, 150),    # 어두운 노랑 (Dark Yellow)
        (0, 255, 0),      # 초록 (Green)
        (0, 100, 0),      # 어두운 초록 (Dark Green)
        (255, 0, 0),      # 파랑 (Blue)
        (139, 0, 0),      # 어두운 파랑 (Dark Blue)
        (255, 255, 255),  # 흰색(white)
        (0, 0, 0)         # 검정(black)
    ]

# ===============================
# 동그라미 생성
# ===============================
def draw_circle(img):
    height, width = img.shape[:2]

    # ===============================
    # 원 중심 → 이미지 정중앙
    # ===============================
    circle_x = width // 2
    circle_y = height // 2

    radius = random.randint(70, 100)

    mask = np.zeros((height, width), dtype=np.uint8)

    color = random.choice(colors)

    # cv2.circle(img, (circle_x, circle_y), radius, random.choice(colors), -1)
    # cv2.circle(mask, (circle_x, circle_y), radius, 255, -1)
    cv2.circle(img, (circle_x, circle_y), radius, color, -1)
    cv2.circle(mask, (circle_x, circle_y), radius, color, -1)

    return img, mask, (circle_x, circle_y, radius * 2, radius * 2)

# # ===============================
# # + 교차로 생성
# # ===============================
# def draw_cross(img):
#     cross_x = IMG_SIZE // 2
#     cross_y = IMG_SIZE // 2
#
#     length = random.randint(180, 230)
#     thickness = random.randint(40, 50) # 막대의 두께
#
#     half_thickness = thickness // 2
#
#     mask = np.zeros(img.shape[:2], dtype=np.uint8)
#     # 색상 & 채움 여부 랜덤
#     color = random.choice(colors)
#     fill = random.choice(fills)
#
#     # 교차로 외곽을 구성하는 12개의 점 좌표 (시계 방향)
#     pts = np.array([
#         [cross_x - half_thickness, cross_y - length], [cross_x + half_thickness, cross_y - length],         # 위쪽 날개 끝
#         [cross_x + half_thickness, cross_y - half_thickness], [cross_x + length, cross_y - half_thickness], # 오른쪽 날개 위
#         [cross_x + length, cross_y + half_thickness], [cross_x + half_thickness, cross_y + half_thickness], # 오른쪽 날개 아래
#         [cross_x + half_thickness, cross_y + length], [cross_x - half_thickness, cross_y + length],         # 아래쪽 날개 끝
#         [cross_x - half_thickness, cross_y + half_thickness], [cross_x - length, cross_y + half_thickness], # 왼쪽 날개 아래
#         [cross_x - length, cross_y - half_thickness], [cross_x - half_thickness, cross_y - half_thickness]  # 왼쪽 날개 위
#     ], np.int32)
#
#     # ==========================================
#     # -1 내부 채움 / 나머지 테두리만
#     # ==========================================
#     if fill == -1:
#         # 내부를 꽉 채운 교차로
#         cv2.fillPoly(img, [pts], color) # fillPoly 다각형의 내부를 채우는 함수
#         cv2.fillPoly(mask, [pts], 255)
#     else:
#         # 테두리만 있는 교차로 (중앙에 선 안 생김)
#         cv2.polylines(img, [pts], isClosed=True, color=color, thickness=fill)
#         cv2.polylines(mask, [pts], isClosed=True, color=255, thickness=fill)
#
#     return img, mask, (cross_x, cross_y, length * 2, length * 2)

# ===============================
# 경계 파괴 , 도형 경계를 바닥과 섞어버림
# ===============================
# 배경과 도형이 따로 놀지 않게끔 경계를 파괴함.
def break_edges(img, mask):
    blur = cv2.GaussianBlur(mask, (9,9), 0) # (9,9) 블러의 강도 => 클수록 뭉개짐
    alpha = blur.astype(float) / 255

    for c in range(3): # B,G,R 하나씩 처리하기 위함
        img[:,:,c] = img[:,:,c] * (1 - alpha*0.3)   # alpha * 0.3 =>  하얀 부분(1.0)에서 30프로 효과

    return img.astype(np.uint8)

# ===============================
# 카메라 느낌 파괴
# ===============================
def camera_damage(img):
    # 노이즈
    noise = np.random.normal(0, 12, img.shape).astype(np.int32)
    img = np.clip(img + noise, 0, 255).astype(np.uint8) # 0보다 작은 값은 전부 0, 255보다 큰 값은 전부 255

    # 블러
    if random.random() < 0.7:
        img = cv2.GaussianBlur(img, (5,5), random.uniform(0.8,1.4))

    # JPEG 압축 손실
    # 이미지 저장할 때 품질 결정
    cv2.imwrite("tmp.jpg", img, [cv2.IMWRITE_JPEG_QUALITY, random.randint(45,70)])
    img = cv2.imread("tmp.jpg")

    return img

# ===============================
# YOLO 라벨 저장
# ===============================
def save_label(path, class_id, bbox):
    cx, cy, w, h = bbox

    x = cx / IMG_SIZE
    y = cy / IMG_SIZE
    w = w / IMG_SIZE
    h = h / IMG_SIZE

    with open(path, "w") as f:
        f.write(f"{class_id} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

# ===============================
# 학습 데이터 만들기
# ===============================
TOTAL_COUNT = NUM_PER_CLASS
TRAIN_COUNT = int(TOTAL_COUNT * 0.8)    # 80%
VAL_COUNT   = TOTAL_COUNT - TRAIN_COUNT # 20%

for class_id in [0]:
    index = 1   # 클래스가 바뀔 때마다 index를 1로 초기화

    # 클래스당 8:2 비율 계산
    TRAIN_COUNT = int(NUM_PER_CLASS * 0.8)
    VAL_COUNT = NUM_PER_CLASS - TRAIN_COUNT
    # TRAIN 데이터를 1 ~ 160번까지 생성
    for i in range(TRAIN_COUNT):
        img = load_real_background()
        # if class_id == 0:
        img, mask, bbox = draw_circle(img)
        # else:
        #     img, mask, bbox = draw_cross(img)

        # 이미지 처리 및 효과 적용
        img = break_edges(img, mask)    # 경계 파괴
        img = camera_damage(img)        # 카메라 데미지

        # 파일명: circle_1.jpg ~ circle_160.jpg (또는 cross_1.jpg ~ cross_160.jpg)
        img_name = f"{CLASS_NAMES[class_id]}_{index}.jpg"
        cv2.imwrite(IMG_TRAIN + img_name, img)
        save_label(LBL_TRAIN + img_name.replace(".jpg", ".txt"), class_id, bbox)

        index += 1 # 1씩 증가

    # VAL 데이터를 161 ~ 200번까지 생성
    # index가 161부터 시작
    for i in range(VAL_COUNT):
        img = load_real_background()
        # if class_id == 0:
        img, mask, bbox = draw_circle(img)
        # else:
        #     img, mask, bbox = draw_cross(img)

        img = break_edges(img, mask)
        img = camera_damage(img)

        # 파일명: circle_161.jpg ~ circle_200.jpg (또는 cross_161.jpg ~ cross_200.jpg)
        img_name = f"{CLASS_NAMES[class_id]}_{index}.jpg"
        cv2.imwrite(IMG_VAL + img_name, img)
        save_label(LBL_VAL + img_name.replace(".jpg", ".txt"), class_id, bbox)

        index += 1 # 1씩 증가

print("YOLO 학습용 데이터 생성 완료")

YOLO 학습용 데이터 생성 완료


<h1>데이터 학습 시키기</h1>

In [ ]:
from ultralytics import YOLO
import os

def train_yolo():
    # ===============================
    # YOLO 모델 불러오기
    # yolov8s.pt : 사전 학습된 작은 YOLOv8 모델
    # ===============================
    model = YOLO('yolov8m.pt')  # 사용할 가중치 모델 선택
    # dataset_yaml

    # ===============================
    # 학습 실행
    # ===============================
    results = model.train(
        data='C:/Users/User/Documents/GitHub/AI-_-_20260119/KMJ/data.yaml',        # 학습할 데이터셋 정보가 담긴 YAML 파일 경로
        epochs=10,                                          # 학습 반복 횟수
        imgsz=640,                                          # 입력 이미지 크기 (640x640)
        perspective=0.0002,                                 # 원근 변환
        batch=4,                                            # 한 번에 학습할 이미지 수
        workers=0,                                          # CPU 환경 멈춤 방지 필수 설정
        project='legosProject',                             # 학습 결과를 저장할 상위 폴더
        name='trainData',                                   # project 폴더 안에서 결과를 저장할 하위 폴더
        device='cpu',                                       # GPU 번호, CPU는 'cpu' 입력
        verbose=True,                                       # 학습 진행 정보 콘솔 출력 여부
        exist_ok=True,                                      # 기존 동일 이름 폴더가 있어도 덮어쓰기 허용
        augment=True,                                       # 데이터 증강 적용 여부 (회전, 뒤집기 등)
        augmentations='albumentations.yaml',                # albumentations 라이브러리를 이용한 커스텀 증강 설정
        iou=0.5                                             # IOU threshold, 학습 중 Positive/Negative 판단 기준
    )

    # 학습이 끝나면 results 객체에 loss, mAP, precision, recall 등 학습 지표가 담김
    return results

if __name__ == '__main__':
    results = train_yolo()
    print("학습 완료")
    print(results)

<h1> 정확도 확인 </h1>

In [1]:
from ultralytics import YOLO
import numpy as np
import cv2
import os

# ===============================
# 모델 불러오기
# ===============================
model = YOLO("legosProject/trainData/weights/best.pt")

# ===============================
# 특정 이미지 경로
# ===============================
image_path = "yolo_dataset/test/images/test_1.jpg"

# ===============================
# 예측 수행
# ===============================
# runs/detect/predict 폴더에 생성됨.
result = model.predict(source=image_path, imgsz=640, save=True, conf=0.1, exist_ok=True)[0]

# result.boxes → 예측 박스
pred_boxes = result.boxes.xyxy.cpu().numpy()   # [x1, y1, x2, y2]
pred_classes = result.boxes.cls.cpu().numpy()  # 클래스 번호

print("예측 박스 좌표:", pred_boxes)
print("예측 클래스:", pred_classes)

# ===============================
# 실제 라벨 읽기 (YOLO TXT 형식)
# ===============================
label_path = image_path.replace("images", "labels").replace(".jpg", ".txt")
true_boxes = []
true_classes = []

# 경로에 존재하는지 확인
if os.path.exists(label_path):
    with open(label_path, "r") as f:
        for line in f.readlines():
            parts = line.strip().split()
            cls = int(parts[0])
            x_center, y_center, width, height = map(float, parts[1:])

            # YOLO normalized coords → xyxy 픽셀 coords
            img = cv2.imread(image_path)
            img_height, image_weight = img.shape[:2]
            x1 = (x_center - width/2) * image_weight
            y1 = (y_center - height/2) * img_height
            x2 = (x_center + width/2) * image_weight
            y2 = (y_center + height/2) * img_height

            true_boxes.append([x1, y1, x2, y2])
            true_classes.append(cls)

true_boxes = np.array(true_boxes)
true_classes = np.array(true_classes)

print("실제 라벨 박스:", true_boxes)
print("실제 클래스:", true_classes)

# ===============================
# IoU 계산 함수
# ===============================
def iou(box1, box2):
    # box: [x1, y1, x2, y2]
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    interArea = max(0, x2 - x1) * max(0, y2 - y1)
    box1Area = (box1[2]-box1[0])*(box1[3]-box1[1])
    box2Area = (box2[2]-box2[0])*(box2[3]-box2[1])

    return interArea / (box1Area + box2Area - interArea + 1e-6) # 에러 방지하려고 1e-6을 더한다.

# ===============================
# 일치율 계산
# ===============================
matches = 0
for i, t_cls in enumerate(true_classes):
    for j, p_cls in enumerate(pred_classes):
        if t_cls == p_cls and iou(true_boxes[i], pred_boxes[j]) >= 0.5:  # IoU 0.5 기준
            matches += 1
            break

accuracy = matches / len(true_classes) if len(true_classes) > 0 else 0
print(f"일치한 객체 수: {matches}/{len(true_classes)}, 일치율: {accuracy*100:.2f}%")



image 1/1 C:\Users\User\Documents\GitHub\AI-_-_20260119\KMJ\yolo_dataset\test\images\test_1.jpg: 480x640 1 circle, 141.0ms
Speed: 1.5ms preprocess, 141.0ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
Results saved to C:\Users\User\Documents\GitHub\AI-_-_20260119\KMJ\runs\detect\predict
예측 박스 좌표: [[      120.1       103.3      497.02      379.49]]
예측 클래스: [          0]
실제 라벨 박스: [[        116          98         504         384]]
실제 클래스: [0]
일치한 객체 수: 1/1, 일치율: 100.00%


<h1> 사람 학습 데이터(200건) 생성 </h1>

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F  # 필터링을 위한 F 객체 임포트

Cname = "person"
target_num = 200  # 최종적으로 필요한 장수
load_num = 500    # 서버에서 넉넉하게 가져올 장수

# 1. 넉넉하게 로드 (이미 다운로드된 게 있다면 금방 끝납니다)
dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="train",
    label_types=["detections"],
    classes=[Cname],
    max_samples=load_num,
)

# 2. 필터링 + 셔플 + 타겟 수량만큼 선택
# view는 원본 데이터를 복사하지 않고 가상으로 보여주기만 하므로 메모리 걱정은 안 하셔도 됩니다.
final_view = (
    dataset
    .filter_labels("ground_truth", F("label") == Cname)
    .shuffle()
    .take(target_num)
)

# 3. 내보내기
final_view.export(
    export_dir=f"./coco_{Cname}_{target_num}",
    dataset_type=fo.types.YOLOv5Dataset,
    label_field="ground_truth",
    classes=[Cname],
    overwrite=True
)

print(f"전체 {load_num}개 중 무작위로 {target_num}개의 {Cname} 데이터를 추출 완료했습니다.")

<h1> 레이블 수정 2로 수정해주어야한다.</h1>
<h3> 기본적으로 0클래스로 제공하므로 2로 변경해주어야한다.</h3>

In [ ]:
import os
import glob

# 데이터가 들어있는 폴더 경로
label_files = glob.glob('C:/Users/User/Documents/GitHub/AI-_-_20260119/KMJ/coco_person_200/labels/val/*.txt')

for file_path in label_files:
    with open(file_path, 'r') as f:
        lines = f.readlines()

    with open(file_path, 'w') as f:
        for line in lines:
            parts = line.split()
            if len(parts) > 0:
                # 무조건 클래스 번호를 2로 변경 (사람)
                parts[0] = '2'
                f.write(" ".join(parts) + "\n")

print(f"총 {len(label_files)}개의 라벨 수정 완료! 이제 2번 클래스로 학습 가능합니다.")